In [1]:
import copernicusmarine
import xarray as xr
import fsspec
import pandas as pd

## Load DATA


### Observations


In [2]:
url = "https://raw.githubusercontent.com/Ash12H/seapopym-data/dev/data/bats/3_post_processed/bats_product.zarr"
bats_obs = xr.open_dataset(fsspec.get_mapper(url), engine="zarr")
bats_obs

<xarray.Dataset> Size: 15MB
Dimensions:                        (time: 4242, depth: 8, latitude: 1,
                                    longitude: 1, sieve_size: 5)
Coordinates:
  * depth                          (depth) float64 64B 50.0 100.0 ... 400.0
  * latitude                       (latitude) float64 8B 31.6
  * longitude                      (longitude) float64 8B -64.2
  * sieve_size                     (sieve_size) float64 40B 0.2 0.5 1.0 2.0 5.0
  * time                           (time) datetime64[ns] 34kB 1988-10-21T04:5...
Data variables: (12/39)
    CO2                            (time, depth, latitude, longitude) float64 271kB ...
    CTD_salinity                   (time, depth, latitude, longitude) float64 271kB ...
    POC                            (time, depth, latitude, longitude) float64 271kB ...
    PON                            (time, depth, latitude, longitude) float64 271kB ...
    POP                            (time, depth, latitude, longitude) float64 271kB ...
    TN                             (time, depth, latitude, longitude) float64 271kB ...
    ...                             ...
    synechococcus                  (time, depth, latitude, longitude) float64 271kB ...
    t0                             (time, depth, latitude, longitude) float64 271kB ...
    temp                           (time, depth, latitude, longitude) float64 271kB ...
    total_dissolved_phosphorus     (time, depth, latitude, longitude) float64 271kB ...
    wet_weight_migrant             (sieve_size, time, latitude, longitude, depth) float64 1MB ...
    wet_weight_resident            (sieve_size, time, latitude, longitude, depth) float64 1MB ...

In [3]:
LATITUDE = bats_obs.latitude.data[0]
LONGITUDE = bats_obs.longitude.data[0]
START_TIME = pd.Timestamp(bats_obs.time.data[0]).strftime("%Y-%m-%d")
END_TIME = pd.Timestamp(bats_obs.time.data[-1]).strftime("%Y-%m-%d")
DELTA = 0.5

print(f"Latitude: {LATITUDE}\nLongitude: {LONGITUDE}\nStart Time: {START_TIME}\nEnd Time: {END_TIME}\nDelta: {DELTA}")

Latitude: 31.6
Longitude: -64.2
Start Time: 1988-10-21
End Time: 2022-12-16
Delta: 0.5


### CMEMS


In [4]:
FROM_API = False

In [5]:
# copernicusmarine.login()

coordinates = {
    "minimum_latitude": LATITUDE - DELTA,
    "maximum_latitude": LATITUDE + DELTA,
    "minimum_longitude": LONGITUDE - DELTA,
    "maximum_longitude": LONGITUDE + DELTA,
    "start_datetime": START_TIME,
    "end_datetime": END_TIME,
}

if FROM_API:
    bio = copernicusmarine.open_dataset(dataset_id="cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i", **coordinates)
    bio.load()
    physic = copernicusmarine.open_dataset(dataset_id="cmems_mod_glo_bgc_my_0.083deg-lmtl-Fphy_PT1D-i", **coordinates)
    physic.load()
else:
    bio = xr.load_dataset("../../data/phd/BATS/CMEMS/cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i_1718163947378.nc")
    physic = xr.load_dataset(
        "../../data/phd/BATS/CMEMS/cmems_mod_glo_bgc_my_0.083deg-lmtl-Fphy_PT1D-i_1718163399766.nc"
    )


bats_data = xr.merge([bio, physic])
bats_data = bats_data.sel(time=slice(START_TIME, END_TIME)).sel(
    latitude=[LATITUDE], longitude=[LONGITUDE], method="nearest"
)
lat_attrs, lon_attrs = bats_data.latitude.attrs, bats_data.longitude.attrs
bats_data = bats_data.assign_coords({"latitude": [LATITUDE], "longitude": [LONGITUDE]})
bats_data.latitude.attrs, bats_data.longitude.attrs = lat_attrs, lon_attrs
bats_data

<xarray.Dataset> Size: 839kB
Dimensions:              (time: 9116, latitude: 1, longitude: 1, depth: 3)
Coordinates:
  * time                 (time) datetime64[ns] 73kB 1998-01-01 ... 2022-12-16
  * depth                (depth) float32 12B 1.0 2.0 3.0
  * latitude             (latitude) float64 8B 31.6
  * longitude            (longitude) float64 8B -64.2
Data variables: (12/13)
    mnkc_epi             (time, latitude, longitude) float32 36kB 0.7599 ... ...
    mnkc_hmlmeso         (time, latitude, longitude) float32 36kB 0.6979 ... ...
    mnkc_lmeso           (time, latitude, longitude) float32 36kB 1.093 ... 1...
    mnkc_mlmeso          (time, latitude, longitude) float32 36kB 0.576 ... 0...
    mnkc_mumeso          (time, latitude, longitude) float32 36kB 0.6176 ... ...
    mnkc_umeso           (time, latitude, longitude) float32 36kB 0.3953 ... ...
    ...                   ...
    zeu                  (time, latitude, longitude) float32 36kB 66.99 ... 8...
    zooc                 (time, latitude, longitude) float32 36kB 0.3641 ... ...
    T                    (time, depth, latitude, longitude) float32 109kB 20....
    U                    (time, depth, latitude, longitude) float32 109kB 0.1...
    V                    (time, depth, latitude, longitude) float32 109kB -0....
    pelagic_layer_depth  (time, depth, latitude, longitude) float32 109kB 100...
Attributes:
    Conventions:       CF-1.11
    title:             Global ocean low and mid trophic levels biomass conten...
    institution:       CLS
    source:            SEAPODYM-LMTL 3.0.0
    history:           Created on 2022-09-28
    references:        http://www.cls.fr; http://www.seapodym.eu
    subset:source:     ARCO data downloaded from the Marine Data Store using ...
    subset:productId:  GLOBAL_MULTIYEAR_BGC_001_033
    subset:datasetId:  cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i_202211
    subset:date:       2024-06-12T03:45:47.386Z

## Gathering Data & Save


In [7]:
bats_data.to_zarr("bats_cmems.zarr", mode="w")
bats_obs.to_zarr("bats_obs.zarr", mode="w")